<a href="https://colab.research.google.com/github/Joshitha18/Language_Diarization/blob/main/seg_fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Audio

In [ ]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5888 sha256=7b9dc03596e41e2a38a8d2bcbfa4f9dfd119ee90c5d89e524b66932c4f93bf2b
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
import python_speech_features
import numpy as np
import soundfile as sf
import pickle
import cv2
import glob

1. Take the 1000 files and compute 200ms (3200) samples for all and save it in .pkl  #chcek all 200ms have 3200 samples
2. Compute mfcc for all 200 ms frames #check all have 19 frames
3. for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

4.t = np.array(tel_feat[0:2467])
t.shape _: samples,19,39
samples: Number of 200 ms segments
time steps: No of frames in it
features :  mfcc

Step 1: Take 1000 files and find 200ms telugu segments in each

In [ ]:
#Make Dataset ready
import csv
import pandas as pd
list = ['File Name','Sequence']
df = pd.read_csv("/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Transcription_LT_Sequence_Frame_Level_200_actual.tsv", delimiter="\t",names=list)
df['File Name'] = df['File Name'].apply(lambda x: str(x))
for x in range(0,len(df['File Name'])):
  df['File Name'][x] = str(str(df['File Name'][x]).zfill(9))
df

,File Name,Sequence
0,000010010,SSTTTTTTTTTTEEEEEETTSEEEEESSSSTTTTTTSSSSSSSSTT...
1,000010012,SSEEEETTSTTTTTTTTTTTTTTTTTTTTTSS
2,000010017,SSEEETTTTTTTTTTTTTTTTTTTTTTS
3,000010023,SSTTTTTTTTTTTTTTTTTTEESSSSSSEEEEEEETTTTTTTTSS
4,000010030,SSTTTTTTTTTTEETTTS
...,...,...
8220,102280462,SSTTTTTTEEEEETTTTTTTSS
8221,102280464,SSTTTTTTEETEETTSS
8222,102280484,SSTTTTTEETTETTTTT
8223,102280489,SSEEETTTTTTTTEETTTSS


Take all Telugu segments

In [ ]:
path = "/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Audio/*.*"

In [ ]:
#Find all telugu 200 ms frames
tel_frames= []
cnt = 0

fin = []
#Run for file name in df
for file_name in df['File Name']:
  cnt = cnt+1
  #Combine name with path
  file_path ="/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/Audio/"+file_name+'.wav'
  #Read the file
  data, fs = sf.read(file_path)
 
  #------Get sequence of Telugu---------#
  seq = df.loc[df['File Name'] == file_name, 'Sequence'].iloc[0]

    #---------Find indexes of Telugu segments in seq-----------------#
  index = []
  for i in range(len(seq)):
      if (seq[i] == 'T'):
          index.append(i)
  #find the speech data from each index and take 200 ms data, i.e 0.2*16000 = 3200
  frame = int(0.2*fs)
  #fin = []
  for i in index:
    if len(data[frame*i:frame*i+frame])!=3200:
      print(cnt," ",file_name," ",i)
    else:
      fin.append((data[frame*i:frame*i+frame]).tolist())
  #APpend in final list
  if cnt>1000:
    break


241   000100027   35
278   000100439   31
286   000110082   38
300   000110253   53
312   000110362   24
347   000130088   42
354   000130201   25
447   000190408   25
493   000230043   29
518   000230309   62
530   000230392   37
645   000300352   22
646   000300366   33


In [ ]:
len(fin)

24678

In [ ]:
print(fin[0])

[0.000701904296875, 0.00067138671875, 0.00103759765625, 0.00189208984375, 0.0025634765625, 0.003509521484375, 0.00445556640625, 0.005462646484375, 0.005584716796875, 0.0032958984375, 0.00140380859375, 0.0010986328125, 0.001068115234375, -0.000579833984375, -0.002716064453125, -0.002899169921875, -0.0008544921875, 0.00103759765625, 0.001708984375, 0.001556396484375, -9.1552734375e-05, -0.00018310546875, 0.000885009765625, 0.001739501953125, 0.002197265625, 0.00213623046875, 0.0028076171875, 0.002685546875, 0.001739501953125, 0.0009765625, 0.0003662109375, -0.000152587890625, 0.00048828125, 0.00067138671875, -0.0001220703125, -0.00091552734375, -0.000701904296875, -6.103515625e-05, 0.0006103515625, 0.001983642578125, 0.001678466796875, 6.103515625e-05, -0.001220703125, -0.00067138671875, 0.000518798828125, 0.00067138671875, -0.000213623046875, -0.003448486328125, -0.00482177734375, -0.002593994140625, 0.000213623046875, 0.0035400390625, 0.004364013671875, 0.002532958984375, -0.0001831054

In [ ]:
for x in fin:

  if len(x)!=3200:
  
    print("Founf")
    print(len(x))
  

MFCC vectors

In [ ]:
final_feat_vec = []
for data in fin:
  data = np.array((data))
  feat_file = []
  mfcc_feat = python_speech_features.mfcc(data,16000,winlen=0.02,winstep=0.01)
  #-------Compute delta coef-----#
  N=2
  delta = python_speech_features.delta(mfcc_feat, 2)
  #--------Compute delta-delta coeff-------#
  delta_delta = python_speech_features.delta(delta, 2)

  for frame in range(0,len(mfcc_feat)):
    #frame opened
    #concatenate for each frame : 13 mfcc + 13 delta + 13 delta-delta = 39 features
    feat_ = np.concatenate((mfcc_feat[frame],delta[frame],delta_delta[frame]))
    feat_file.append(feat_)

  final_feat_vec.append(feat_file)

In [ ]:
for x in final_feat_vec:

  if len(x)!=19:
  
    print("Founf")
    print(len(x))
  

In [ ]:
print(final_feat_vec[0])

[array([-8.28719032e+00, -1.13347611e+01, -3.01328985e+01, -1.02183549e+00,
        5.05976291e+00,  6.01580903e+00,  1.42289013e+01,  1.76303173e+01,
        4.21620537e+00,  5.38378425e+00, -2.38234858e+00,  6.46840059e+00,
        9.40192492e+00, -6.28889692e-02, -9.51486909e-01, -3.42003309e-01,
       -9.88968581e-01, -2.08766134e+00,  3.97873890e-01, -1.34747231e+00,
       -2.45847306e-01,  2.33128228e+00, -2.42901207e-01,  9.74727779e-01,
       -1.00072325e-01, -1.33486978e+00, -1.59122761e-02,  1.37564663e-01,
        1.48142018e-01,  4.96692185e-01,  3.04829931e-01, -3.96158790e-02,
        3.91048911e-01,  3.36084104e-01, -7.19189540e-01, -3.65012013e-02,
        4.59455374e-01,  1.00436816e-02, -6.19904972e-01]), array([-8.36641164e+00, -1.06482045e+01, -2.81928372e+01, -1.83197708e+00,
        3.96222225e+00,  1.42878538e+01,  1.32618103e+01,  1.63838714e+01,
        1.07488195e+01,  6.12976276e+00,  1.26734297e+00,  5.12138307e+00,
        4.78382373e+00, -1.34855932e-01

In [ ]:
#feat_file = open("/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/telugu_feat_1000.pkl", "wb")
#pickle.dump(final_feat_vec, feat_file)
#feat_file.close()

In [ ]:
feat_file = open("/content/gdrive/MyDrive/MS_CD_SW/PartB_Telugu/Train/telugu_feat_1000.pkl", "rb")
tel_feat = pickle.load(feat_file)

In [ ]:
len(tel_feat[0][18])

39

Step 3

In [ ]:
import numpy

In [ ]:
for i in range(0,len(tel_feat)):
  tel_feat[i] = numpy.stack( tel_feat[i], axis=0 )

In [ ]:
print(tel_feat[0])

[[-8.28719032e+00 -1.13347611e+01 -3.01328985e+01 -1.02183549e+00
   5.05976291e+00  6.01580903e+00  1.42289013e+01  1.76303173e+01
   4.21620537e+00  5.38378425e+00 -2.38234858e+00  6.46840059e+00
   9.40192492e+00 -6.28889692e-02 -9.51486909e-01 -3.42003309e-01
  -9.88968581e-01 -2.08766134e+00  3.97873890e-01 -1.34747231e+00
  -2.45847306e-01  2.33128228e+00 -2.42901207e-01  9.74727779e-01
  -1.00072325e-01 -1.33486978e+00 -1.59122761e-02  1.37564663e-01
   1.48142018e-01  4.96692185e-01  3.04829931e-01 -3.96158790e-02
   3.91048911e-01  3.36084104e-01 -7.19189540e-01 -3.65012013e-02
   4.59455374e-01  1.00436816e-02 -6.19904972e-01]
 [-8.36641164e+00 -1.06482045e+01 -2.81928372e+01 -1.83197708e+00
   3.96222225e+00  1.42878538e+01  1.32618103e+01  1.63838714e+01
   1.07488195e+01  6.12976276e+00  1.26734297e+00  5.12138307e+00
   4.78382373e+00 -1.34855932e-01 -7.80031789e-01  4.65989977e-01
   4.46870403e-01 -1.88519525e+00 -1.10102742e-01 -8.79461477e-01
   6.89907526e-02  1.5651

t = np.array(tel_feat[0:2467]) t.shape _: samples,19,39 samples: Number of 200 ms segments time steps: No of frames in it features : mfcc

In [ ]:
#t.shape _: samples,19,39 
#samples: Number of 200 ms segments 
#time steps: No of frames in it 
#features : mfcc

In [ ]:
t = np.array(tel_feat)

In [ ]:
t.shape

(24678, 19, 39)

In [ ]:
sequence = t

In [ ]:
window_length = 19

In [ ]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# define model
n_in = 19
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(19,39)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(39)))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               56000     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 19, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 19, 100)           80400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 19, 39)            3939      
Total params: 140,339
Trainable params: 140,339
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.input

<KerasTensor: shape=(None, 19, 39) dtype=float32 (created by layer 'lstm_2_input')>

In [ ]:
for layer in model.layers:
  print(layer)

In [ ]:
# fit model
model.fit(sequence, sequence, epochs=50, verbose=1, batch_size=457)

Epoch 1/50
54/54 [==============================] - 38s 234ms/step - loss: 234.6727
Epoch 2/50
54/54 [==============================] - 13s 235ms/step - loss: 135.6150
Epoch 3/50
54/54 [==============================] - 13s 237ms/step - loss: 96.2096
Epoch 4/50
54/54 [==============================] - 13s 238ms/step - loss: 83.8155
Epoch 5/50
54/54 [==============================] - 13s 238ms/step - loss: 65.5104
Epoch 6/50
54/54 [==============================] - 13s 239ms/step - loss: 62.1057
Epoch 7/50
54/54 [==============================] - 13s 237ms/step - loss: 60.8782
Epoch 8/50
54/54 [==============================] - 13s 238ms/step - loss: 61.7319
Epoch 9/50
54/54 [==============================] - 13s 237ms/step - loss: 60.1564
Epoch 10/50
54/54 [==============================] - 13s 239ms/step - loss: 58.8575
Epoch 11/50
54/54 [==============================] - 13s 238ms/step - loss: 57.8604
Epoch 12/50
54/54 [==============================] - 13s 239ms/step - loss: 57.2706

In [ ]:
# fit model
model.fit(sequence, sequence, epochs=100, verbose=1, batch_size=457)

Epoch 1/100
54/54 [==============================] - 13s 248ms/step - loss: 32.9485
Epoch 2/100
54/54 [==============================] - 13s 243ms/step - loss: 32.5980
Epoch 3/100
54/54 [==============================] - 13s 244ms/step - loss: 32.2715
Epoch 4/100
54/54 [==============================] - 13s 244ms/step - loss: 31.8381
Epoch 5/100
54/54 [==============================] - 13s 245ms/step - loss: 31.5630
Epoch 6/100
54/54 [==============================] - 13s 245ms/step - loss: 31.2454
Epoch 7/100
54/54 [==============================] - 13s 245ms/step - loss: 31.0186
Epoch 8/100
54/54 [==============================] - 13s 243ms/step - loss: 30.5701
Epoch 9/100
54/54 [==============================] - 13s 243ms/step - loss: 30.4320
Epoch 10/100
54/54 [==============================] - 13s 245ms/step - loss: 30.1838
Epoch 11/100
54/54 [==============================] - 13s 243ms/step - loss: 29.8469
Epoch 12/100
54/54 [==============================] - 13s 242ms/step - los

In [ ]:
# fit model
model.fit(sequence, sequence, epochs=100, verbose=1, batch_size=457)

Epoch 1/100
54/54 [==============================] - 13s 242ms/step - loss: 22.6872
Epoch 2/100
54/54 [==============================] - 13s 242ms/step - loss: 22.6845
Epoch 3/100
54/54 [==============================] - 13s 242ms/step - loss: 22.6080
Epoch 4/100
54/54 [==============================] - 13s 241ms/step - loss: 22.4740
Epoch 5/100
54/54 [==============================] - 13s 240ms/step - loss: 22.5729
Epoch 6/100
54/54 [==============================] - 13s 242ms/step - loss: 22.4183
Epoch 7/100
54/54 [==============================] - 13s 241ms/step - loss: 22.4804
Epoch 8/100
54/54 [==============================] - 13s 243ms/step - loss: 22.3955
Epoch 9/100
54/54 [==============================] - 13s 242ms/step - loss: 22.3144
Epoch 10/100
54/54 [==============================] - 13s 242ms/step - loss: 22.2567
Epoch 11/100
54/54 [==============================] - 13s 240ms/step - loss: 22.3101
Epoch 12/100
54/54 [==============================] - 13s 242ms/step - los

In [ ]:
model.save('wts.h5')

In [ ]:
!gdown --id 1tLMgB6_TYhUa_FGSFW9ski8aWD8U0A_Y
from keras import models    
model = models.load_model("wts.h5")

Downloading...
From: https://drive.google.com/uc?id=1tLMgB6_TYhUa_FGSFW9ski8aWD8U0A_Y
To: /content/wts.h5
100% 1.73M/1.73M [00:00<00:00, 27.3MB/s]


In [ ]:
y_pred = model.predict(sequence, batch_size=457)

In [ ]:
sequence[0][0]

array([-8.28719032e+00, -1.13347611e+01, -3.01328985e+01, -1.02183549e+00,
        5.05976291e+00,  6.01580903e+00,  1.42289013e+01,  1.76303173e+01,
        4.21620537e+00,  5.38378425e+00, -2.38234858e+00,  6.46840059e+00,
        9.40192492e+00, -6.28889692e-02, -9.51486909e-01, -3.42003309e-01,
       -9.88968581e-01, -2.08766134e+00,  3.97873890e-01, -1.34747231e+00,
       -2.45847306e-01,  2.33128228e+00, -2.42901207e-01,  9.74727779e-01,
       -1.00072325e-01, -1.33486978e+00, -1.59122761e-02,  1.37564663e-01,
        1.48142018e-01,  4.96692185e-01,  3.04829931e-01, -3.96158790e-02,
        3.91048911e-01,  3.36084104e-01, -7.19189540e-01, -3.65012013e-02,
        4.59455374e-01,  1.00436816e-02, -6.19904972e-01])

In [ ]:
y_pred[0][0]

array([-7.2482705e+00, -4.5234032e+00, -1.8176981e+01, -9.0496693e+00,
        5.5210037e+00, -6.5669785e+00,  8.0711460e+00,  6.9829683e+00,
        8.5808582e+00,  1.5017054e+00, -3.3944876e+00,  7.6205220e+00,
       -1.2337571e+00,  2.5313872e-01,  8.4068978e-01, -1.2660830e+00,
        7.5048983e-01, -1.2856158e+00,  7.5261605e-01,  2.5712091e-01,
       -6.4772421e-01,  2.8473389e-01,  4.5249768e-02,  1.0233603e+00,
       -3.9935596e-02, -1.2014568e-01,  4.5990307e-02,  1.7395023e-01,
        1.8680377e-01,  1.6548097e-01, -5.5198795e-01,  8.8684037e-03,
       -3.7466565e-01, -5.0600612e-01, -3.8260457e-01, -1.3902609e-01,
        1.4640838e-03, -4.3578416e-01,  8.6151332e-02], dtype=float32)

In [ ]:
out = model.layers[0].output
m = Model(inputs = model.input, outputs=out)

In [ ]:
output = m.predict(sequence)

In [ ]:
len(output[0])

100